Zadanie domowe 4

Zadanie dotyczy algorytmu Support Vector Machine, który był omówiony na zajęciach. Zainteresowanych zachęcamy do zapoznania się z http://pyml.sourceforge.net/doc/howto.pdf.

Wykorzystaj dwa zbiory danych:

    apartments z R-owego pakietu DALEX,
    dowolny, wybrany przez siebie zbiór danych (najlepiej z co najmniej 8 zmiennymi numerycznymi).

    Dopasuj SVM do obu zbiorów danych.
    Sprawdź, czy zalinkowany artykuł słusznie zwraca uwagę na skalowanie danych (pamiętaj, że większość implementacji domyślnie skaluje).
    Spróbuj zoptymalizować metodą random search najważniejsze hiperparametry tj. :

    cost,
    gamma,
    degree, najprościej optymalizować hiperparametry w SVM z jądrem gaussowskim, ale można też poszukać najlepszego jądra.

Zadanie bonusowe za 2 punkty (uwzględnione w całościowej punktacji przedmiotu)

Aby uzyskać bonusowy punkt należy wykorzystać po raz kolejny zbiór danych z PD2 i nauczyć dowolny modele regresji do prognozowania zmiennej price (traktujemy ją teraz jak zmienną celu). W kwestii zmiennych objaśniających należy się ograniczyć do main_category, categories i it_location.

Rozwiązanie powinno zawierać:

    zastosowanie target encodingu (należy przemyśleć parametr smoothing i wyjaśnić dlaczego jest on ważny)
    nauczenie modelu liniowego do przewidywania ceny produktu
    zastosowanie co najmniej dwóch wariantów regularyzacji i przeanalizowanie ich wpływu na jakość predykcji (należy dodać krótki opis zastosowanych metod)
    nauczenie innego modelu regresyjnego
    wykorzystanie miary RMSE i R2 do wybrania najlepszego wariantu modelu


In [1]:
import pandas as pd
import numpy as np
import dalex as dx

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler

import category_encoders as ce


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df = dx.datasets.load_apartments()

In [3]:
df2 = pd.read_csv('heart.csv')
map_dict = {'Present': 1, 'Absent':0}
df2['famhist'] = df2['famhist'].map(map_dict)

In [4]:
df2

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,31.72,0,64,28.45,0.00,58,0
458,182,4.20,4.41,32.10,0,52,28.61,18.72,52,1
459,108,3.00,1.59,15.23,0,40,20.09,26.64,55,0
460,118,5.40,11.61,30.79,0,64,27.35,23.97,40,0


In [5]:
y = np.array(df['district'])
X = df.drop(['district'], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model_apartments = SVC()#kernel='poly' zacina się na strojeniu hiperparametrów

In [8]:
model_apartments.fit(X_train,y_train)

SVC()

In [9]:
parameters = {
    'degree': [1,2,3], #ignoruje dla kernel !
    'C': [1, 3, 6, 10],
    'gamma': ['auto', 'scale']
}

clf_rand = RandomizedSearchCV(model_apartments, parameters, cv=3, n_iter=5)

clf_rand.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=5,
                   param_distributions={'C': [1, 3, 6, 10], 'degree': [1, 2, 3],
                                        'gamma': ['auto', 'scale']})

In [10]:
pd.DataFrame(clf_rand.cv_results_)[['params', 'mean_test_score']].sort_values('mean_test_score', ascending=False)

,params,mean_test_score
1,"{'gamma': 'scale', 'degree': 2, 'C': 1}",0.251251
4,"{'gamma': 'scale', 'degree': 3, 'C': 10}",0.247501
0,"{'gamma': 'auto', 'degree': 1, 'C': 3}",0.115002
2,"{'gamma': 'auto', 'degree': 2, 'C': 1}",0.115002
3,"{'gamma': 'auto', 'degree': 3, 'C': 3}",0.115002


In [11]:
y = np.array(df2['chd'])
X = df2.drop(['chd'], axis=1)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
model_heart = SVC()#kernel='poly'

In [14]:
model_heart.fit(X_train,y_train)

SVC()

In [15]:
parameters = {
    'degree': [1,2,3],
    'C': [1, 3, 6, 10],
    'gamma': ['auto', 'scale']
}

clf_rand = RandomizedSearchCV(model_heart, parameters, cv=3, n_iter=5)

clf_rand.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=5,
                   param_distributions={'C': [1, 3, 6, 10],
                                        'degree': [1, 2, 3, 4, 5],
                                        'gamma': ['auto', 'scale']})

In [16]:
pd.DataFrame(clf_rand.cv_results_)[['params', 'mean_test_score']].sort_values('mean_test_score', ascending=False)

,params,mean_test_score
0,"{'gamma': 'scale', 'degree': 3, 'C': 6}",0.674797
3,"{'gamma': 'scale', 'degree': 2, 'C': 6}",0.674797
1,"{'gamma': 'auto', 'degree': 1, 'C': 10}",0.647696
2,"{'gamma': 'auto', 'degree': 2, 'C': 3}",0.647696
4,"{'gamma': 'auto', 'degree': 5, 'C': 3}",0.647696


In [17]:
model_opt = SVC(gamma='scale',degree=1,C=1)

In [18]:
cross_val_score(model_opt, X_train, y_train, cv=3).mean()

0.6504065040650406

In [19]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)


In [20]:
cross_val_score(model_opt, X_train, y_train, cv=3).mean()

0.7235772357723578

# Jak widać po przeskalowaniu model się poprawił